In [1]:
from typing import List
from pydantic import BaseModel
from crewai import Agent, Crew
from crewai.tasks.conditional_task import ConditionalTask # this is the first i'm using this feature
from crewai.tasks.task_output import TaskOutput
from crewai.task import Task
from crewai_tools import SerperDevTool
from crewai import LLM

from crewai.tools import tool
import re
from dotenv import load_dotenv
import json

load_dotenv()

True

In [2]:
class QuestionOutput(BaseModel):
    question: list[str]


class AnswerOutput(BaseModel):
    answers: list[dict]  # List of {question, answer} pairs

In [3]:

llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.2
)

In [ ]:
# Define a condition function for the conditional task
# Returns True if task should execute, False if it should skip
def should_answer_questions(output: TaskOutput) -> bool:
    """Check if exactly 10 questions were generated"""
    if not output.pydantic:
        return False
    question_count = len(output.pydantic.question)
    print(f"\n[CONDITION CHECK] Questions generated: {question_count}")
    return question_count == 11  # Execute only if exactly 10 questions # false


In [9]:
# Define the agents
question_creator = Agent(
    role="Expert Interview Question Generator",
    goal="Generate exactly 10 technical interview questions that assess deep knowledge and problem-solving skills",
    backstory="You are an experienced technical interviewer with 10+ years of expertise. "
    "You understand what makes effective interview questions - they should be challenging, "
    "specific to the domain, and help assess both theoretical knowledge and practical understanding. "
    "Your questions are designed to differentiate between candidates and identify true expertise.",
    llm=llm,
    verbose=True
)

question_answer = Agent(
    role="Expert Technical Interviewer and Answer Provider",
    goal="Provide comprehensive, precise, and concise answers to technical interview questions that would impress an interviewer",
    backstory="You are a seasoned technical expert and interviewer with deep knowledge across multiple domains. "
    "You provide answers that demonstrate mastery, practical experience, and the ability to communicate complex concepts clearly. "
    "Your answers are structured, include relevant examples, and cover both theoretical foundations and real-world applications.",
    llm=llm,
    verbose=True
)


In [10]:
task1 = Task(
    name="Generate Technical Interview Questions",
    description="""You are an expert in the {domain} domain. 
    
    Your task is to generate EXACTLY 10 technical interview questions that:
    - Are appropriate for assessing senior-level expertise
    - Cover different aspects of {domain} (fundamentals, advanced concepts, practical applications)
    - Include both theoretical questions and scenario-based/practical questions
    - Are challenging enough to differentiate expert candidates
    - Follow the style and format of real technical interviews at top companies
    
    Format your response as a dict list with exactly 10 questions.
    Each question should be clear, specific, and focused on one concept.""",
    expected_output="""return the response with this exact structure:
        "question": [
            "Question 1: [specific technical question]",
            "Question 2: [specific technical question]",
            ... (exactly 10 questions total)
        ]""",
    agent=question_creator,
    output_pydantic=QuestionOutput,
    max_retries=2
)

conditional_task = ConditionalTask(
    name="Answer Technical Interview Questions",
    description="""You have received a list of {domain} technical interview questions.
    
    Your task is to provide high-quality, interview-ready answers that:
    - Are precise and concise (2-4 bullet points per answer typically)
    - Include key concepts, examples, or metrics where relevant
    - Demonstrate both depth and clarity of understanding
    - Cover practical implications alongside theoretical knowledge
    - Are structured in a way that impresses technical interviewers
    
    For each question in the provided list, generate a comprehensive answer.""",
    expected_output="""For each question, provide a structured answer in this format:
    
    Q1: [Original Question]
    * [Key point 1 - main concept/definition]
    * [Key point 2 - important details or examples]
    
    Q2: [Original Question]
    * [Continue this format for all questions]
    
    Ensure each answer is professional, technical, and suitable for an interview setting.""",
    condition=should_answer_questions, # this is the first time i'm using this feature
    agent=question_answer,
    output_pydantic=AnswerOutput,
)



In [11]:
# Create a crew with the tasks
crew = Crew(
    agents=[question_creator, question_answer],
    tasks=[task1, conditional_task],
    verbose=True,
)


In [12]:
# Run the crew with a specific domain
result = crew.kickoff(inputs={"domain": "Machine Learning"})
print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)
print(result)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  366ea8e4-f33f-451c-ae00-23681a473403                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Generate Technical Interview Questions                                                                   │
│  ID: 17372858-8d12-4bee-93fb-d6bf5acc825d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Interview Question Generator                                                                     │
│                                                                                                                 │
│  Task: You are an expert in the Machine Learning domain.                                                        │
│                                                                                                                 │
│      Your task is to generate EXACTLY 10 technical interview questions that:                                    │
│      - Are appropriate for assessing senior-level expertise                                                     │
│      - Cover different aspects of Machine Learning (fundamentals, advanced concepts, practical applications)    │
│      - Include both theoretical questions and scenario-based/practical questions                                │
│      - Are challenging enough to differentiate expert candidates                                                │
│      - Follow the style and format of real technical interviews at top companies                                │
│                                                                                                                 │
│      Format your response as a dict list with exactly 10 questions.                                             │
│      Each question should be clear, specific, and focused on one concept.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Interview Question Generator                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "question": [                                                                                                │
│      "Question 1: Describe the differences between supervised, unsupervised, and reinforcement learning, and    │
│  provide examples of each type of learning in real-world applications.",                                        │
│      "Question 2: How would you implement a gradient boosting algorithm from scratch, and what are the key      │
│  hyperparameters that need to be tuned for optimal performance?",                                               │
│      "Question 3: Compare and contrast the strengths and weaknesses of different neural network architectures,  │
│  such as convolutional neural networks, recurrent neural networks, and transformers.",                          │
│      "Question 4: Suppose you are given a dataset with high-dimensional features and a small number of          │
│  samples. What techniques would you use to reduce the dimensionality of the data and prevent overfitting?",     │
│      "Question 5: Explain the concept of regularization in machine learning, and describe how different types   │
│  of regularization, such as L1 and L2 regularization, can be used to prevent overfitting.",                     │
│      "Question 6: You are given a time series forecasting problem, and you need to predict the future values    │
│  of a continuous variable. Describe how you would approach this problem using a combination of machine          │
│  learning and statistical techniques.",                                                                         │
│      "Question 7: How would you handle class imbalance in a classification problem, where one class has a       │
│  significantly larger number of instances than the other classes?",                                             │
│      "Question 8: Describe the concept of attention mechanisms in deep learning, and explain how they can be    │
│  used to improve the performance of sequence-to-sequence models.",                                              │
│      "Question 9: Suppose you are given a large dataset with missing values, and you need to impute the         │
│  missing values using a machine learning approach. Describe how you would approach this problem, and what       │
│  techniques you would use to evaluate the performance of your imputation model.",                               │
│      "Question 10: Explain the concept of transfer learning in deep learning, and describe how you would use    │
│  pre-trained models and fine-tuning to adapt a model to a new task or dataset."                                 │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Generate Technical Interview Questions                                                                         │
│  Agent:                                                                                                         │
│  Expert Interview Question Generator                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


[CONDITION CHECK] Questions generated: 10

[2026-02-26 11:55:13][DEBUG]: Skipping conditional task: You have received a list of Machine Learning technical interview questions.

    Your task is to provide high-quality, interview-ready answers that:
    - Are precise and concise (2-4 bullet points per answer typically)
    - Include key concepts, examples, or metrics where relevant
    - Demonstrate both depth and clarity of understanding
    - Cover practical implications alongside theoretical knowledge
    - Are structured in a way that impresses technical interviewers

    For each question in the provided list, generate a comprehensive answer.


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  366ea8e4-f33f-451c-ae00-23681a473403                                                                           │
│  Final Output: {                                                                                                │
│    "question": [                                                                                                │
│      "Question 1: Describe the differences between supervised, unsupervised, and reinforcement learning, and    │
│  provide examples of each type of learning in real-world applications.",                                        │
│      "Question 2: How would you implement a gradient boosting algorithm from scratch, and what are the key      │
│  hyperparameters that need to be tuned for optimal performance?",                                               │
│      "Question 3: Compare and contrast the strengths and weaknesses of different neural network architectures,  │
│  such as convolutional neural networks, recurrent neural networks, and transformers.",                          │
│      "Question 4: Suppose you are given a dataset with high-dimensional features and a small number of          │
│  samples. What techniques would you use to reduce the dimensionality of the data and prevent overfitting?",     │
│      "Question 5: Explain the concept of regularization in machine learning, and describe how different types   │
│  of regularization, such as L1 and L2 regularization, can be used to prevent overfitting.",                     │
│      "Question 6: You are given a time series forecasting problem, and you need to predict the future values    │
│  of a continuous variable. Describe how you would approach this problem using a combination of machine          │
│  learning and statistical techniques.",                                                                         │
│      "Question 7: How would you handle class imbalance in a classification problem, where one class has a       │
│  significantly larger number of instances than the other classes?",                                             │
│      "Question 8: Describe the concept of attention mechanisms in deep learning, and explain how they can be    │
│  used to improve the performance of sequence-to-sequence models.",                                              │
│      "Question 9: Suppose you are given a large dataset with missing values, and you need to impute the         │
│  missing values using a machine learning approach. Describe how you would approach this problem, and what       │
│  techniques you would use to evaluate the performance of your imputation model.",                               │
│      "Question 10: Explain the concept of transfer learning in deep learning, and describe how you would use    │
│  pre-trained models and fine-tuning to adapt a model to a new task or dataset."                                 │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                       

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


FINAL RESULTS
question=['Question 1: Describe the differences between supervised, unsupervised, and reinforcement learning, and provide examples of each type of learning in real-world applications.', 'Question 2: How would you implement a gradient boosting algorithm from scratch, and what are the key hyperparameters that need to be tuned for optimal performance?', 'Question 3: Compare and contrast the strengths and weaknesses of different neural network architectures, such as convolutional neural networks, recurrent neural networks, and transformers.', 'Question 4: Suppose you are given a dataset with high-dimensional features and a small number of samples. What techniques would you use to reduce the dimensionality of the data and prevent overfitting?', 'Question 5: Explain the concept of regularization in machine learning, and describe how different types of regularization, such as L1 and L2 regularization, can be used to prevent overfitting.', 'Question 6: You are given a time serie

In [13]:
response = result.raw

In [14]:
response

'{\n  "question": [\n    "Question 1: Describe the differences between supervised, unsupervised, and reinforcement learning, and provide examples of each type of learning in real-world applications.",\n    "Question 2: How would you implement a gradient boosting algorithm from scratch, and what are the key hyperparameters that need to be tuned for optimal performance?",\n    "Question 3: Compare and contrast the strengths and weaknesses of different neural network architectures, such as convolutional neural networks, recurrent neural networks, and transformers.",\n    "Question 4: Suppose you are given a dataset with high-dimensional features and a small number of samples. What techniques would you use to reduce the dimensionality of the data and prevent overfitting?",\n    "Question 5: Explain the concept of regularization in machine learning, and describe how different types of regularization, such as L1 and L2 regularization, can be used to prevent overfitting.",\n    "Question 6: Y

In [39]:
import json 

if isinstance(response,str):
    parsed_response = json.loads(response)
    result = parsed_response['answers']
    for res in result:
        # for key,value in res.items():
        #     print(f"question :{key}")
        #     print(f"and answer: {value}")
        #     print()
        print(res)

else:
    print(response)


{'Q1': 'Describe the differences between supervised, unsupervised, and reinforcement learning, and provide examples of each type of learning in real-world applications.', 'key_points': ['Supervised learning involves training a model on labeled data to make predictions on new, unseen data, such as image classification or sentiment analysis.', 'Unsupervised learning involves training a model on unlabeled data to discover patterns or relationships, such as clustering or dimensionality reduction.', 'Reinforcement learning involves training an agent to take actions in an environment to maximize a reward, such as game playing or robotics.']}
{'Q2': 'How would you implement a neural network from scratch in Python to solve a binary classification problem, and what are the key considerations for choosing the number of hidden layers and neurons?', 'key_points': ['To implement a neural network from scratch in Python, you would need to define the architecture, including the number of input and out